In [ ]:
pip install cx-copilot

In [8]:
from cx_copilot import OpenAIEmbeddingBlock, PineconeVectorDBBlock, GPTCompletionBlock, CXCopilot

embedding = OpenAIEmbeddingBlock(open_ai_key='YOUR_OPENAI_KEY')
database = PineconeVectorDBBlock("YOUR_PINECONE_KEY", "YOUR_PINECONE_DATACENTER")
gpt = GPTCompletionBlock(open_ai_key = 'YOUR_OPENAI_KEY')

In [10]:
def summarize_ticket(ticket):
    prompt_ticket_summarize = """ticket: Hey, I am a substack writer for over a year and a big fan of the platform. Even though my Substack is paid, I would like to offer free subscriptions to some of my users like college students. Can you help me with that?
    question: How do I offer free subscriptions to some of my users like college students?

    ticket: Hello, I am the writer of a substack called Political Polarization and a bunch of readers are public policy students. I think it would be great for me to conduct a poll among my readers to see what kind of topics they want me to cover in the upcoming versions of Substack. Can you show me how to do that in my next Substack?
    question: How do I add a poll for my readers in my Substack?

    ticket: Hi there, I am a big reader on the substack platform and a fan of a bunch of writers who write here. Unfortunately, I am running on a tight budget as I am in between jobs and would like to temporarily cancel my subscription. Can you help me with that?
    question: How can I pause a subscription?

    ticket: {ticket}
    question:""".format(ticket=ticket)

    return gpt.get_completion(prompt=prompt_ticket_summarize, max_tokens=100, temperature=0.3)

In [11]:
def create_hypothetical_answer(question):
    prompt_hypothetical_answer = """Write a passage to answer the following question from a user:
    Question: {question}
    Answer: """.format(question=question)

    return gpt.get_completion(prompt=prompt_hypothetical_answer, max_tokens=400, temperature=0.5)

In [12]:
def answer_ticket(question="", relevant_info=""):
    prompt_ticket_answer = """ You are a  customer support agent for a company called Substack. Substack lets independent writers and podcasters publish directly to their audience and get paid through subscriptions.
    Answer the following Question based the relevant information provided to you. Make sure your answer has a friendly tone. 
    Please answer the question truthfully and accurately using the relevant information provided. If you do not have enough information to answer the question, please ask the user for more information that would help you answer the question.
    Question: {question}

    Relevant Information:
    {relevant_info}

    Answer:""".format(question=question, relevant_info=relevant_info)

    return gpt.get_completion(prompt=prompt_ticket_answer, max_tokens=500, temperature=0.1)

In [13]:
def get_relevant_information(question, hypothetical_answer):
    embedded_question = embedding.embed_text(question)
    top_k_question = database.lookup(embedded_question, 'substack-answers', top_k=2).ClosestEmbeddings

    embedded_answer = embedding.embed_text(hypothetical_answer)
    top_k_answer = database.lookup(embedded_answer, 'substack-answers', top_k=2).ClosestEmbeddings

    return {
        "Example_1" : {
            "Score": top_k_question[0].score,
            "Question" : top_k_question[0].metadata['question'],
            "Answer": top_k_question[0].metadata['value']
        },
        "Example_2" : {
            "Score": top_k_question[1].score,
            "Question" : top_k_question[1].metadata['question'],
            "Answer": top_k_question[1].metadata['value']
        },
        "Example_3" : {
            "Score": top_k_answer[0].score,
            "Question" : top_k_answer[0].metadata['question'],
            "Answer": top_k_answer[0].metadata['value']
        },
        "Example_4" : {
            "Score": top_k_answer[1].score,
            "Question" : top_k_answer[1].metadata['question'],
            "Answer": top_k_answer[1].metadata['value']
        }
    }


In [18]:
def print_relevant_information(relevant_info):
    print("\n [1]: ")
    print("Score: ", relevant_info['Example_1']['Score'])
    print("Question: ", relevant_info['Example_1']['Question'])
    print("Answer: ", relevant_info['Example_1']['Answer'])
    print("\n [2]: ")
    print("Score: ", relevant_info['Example_2']['Score'])
    print("Question: ", relevant_info['Example_2']['Question'])
    print("Answer: ", relevant_info['Example_2']['Answer'])
    print("\n [3]: ")
    print("Score: ", relevant_info['Example_3']['Score'])
    print("Question: ", relevant_info['Example_3']['Question'])
    print("Answer: ", relevant_info['Example_3']['Answer'])
    print("\n [4]: ")
    print("Score: ", relevant_info['Example_4']['Score'])
    print("Question: ", relevant_info['Example_4']['Question'])
    print("Answer: ", relevant_info['Example_4']['Answer'])

In [19]:
ticket = "How do I offer free subscriptions to some of my users like college students?"

summarized_ticket = summarize_ticket(ticket)
hypothetical_answer = create_hypothetical_answer(summarized_ticket)
relevant_info = get_relevant_information(summarized_ticket, hypothetical_answer)
answer = answer_ticket(summarized_ticket, relevant_info)


print("Answer: ", answer)

print("\n\n Citations: ", print_relevant_information(relevant_info))

Answer:   Hi there! You can offer complimentary paid subscriptions to college students from your publication's Settings page. 
    1. Scroll down to the Payments section and next to "Give away paid subscription for free", select "Add subscribers". 
    2. Enter the email address of the college student you'd like to give a complimentary paid subscription to and select the length of their subscription. You can also add a list of emails to import multiple subscribers at once. 
    3. Click "Add" and they'll be automatically added as a subscriber to your publication. 
    If you have any other questions, please let me know.

 [1]: 
Score:  0.85459882
Question:  How do I offer a complimentary subscription to a reader?
Answer:  You can offer complimentary paid subscriptions to readers from your publication's Settings page.

1. Scroll down to the Payments section and next to "Give away paid subscription for free", select "Add subscribers".

2. Enter the email address of the reader you'd like 